In [1]:
print("Input dataset name:")
dataset = raw_input()
#dataset = 'HTo2LongLivedTo4b_MH-125_MFF-25_CTau-15000mm_TuneCP5_14TeV_pythia8'

Input dataset name:
Data_pre-selected


In [ ]:
box = False
#Setting box=True will only consider signal events that decay right in front of a chamber to be in acceptance

In [2]:
from ROOT import TFile, TTree
from ROOT import gROOT, AddressOf
import uproot
from array import array
from tqdm import tqdm
import numpy as np

Welcome to JupyROOT 6.14/09


In [3]:
comp_ME11 = array('f', [0.])
comp_ME12 = array('f', [0.])
comp_ME13 = array('f', [0.])
comp_ME21 = array('f', [0.])
comp_ME22 = array('f', [0.])
comp_ME31 = array('f', [0.])
comp_ME32 = array('f', [0.])
comp_ME41 = array('f', [0.])
comp_ME42 = array('f', [0.])

wire_ME11 = array('f', [0.])
wire_ME12 = array('f', [0.])
wire_ME13 = array('f', [0.])
wire_ME21 = array('f', [0.])
wire_ME22 = array('f', [0.])
wire_ME31 = array('f', [0.])
wire_ME32 = array('f', [0.])
wire_ME41 = array('f', [0.])
wire_ME42 = array('f', [0.])


In [4]:
in_path = "/uscms/home/menendez/nobackup/Trigger/CMSSW_10_6_4/src/Data/TPEHists_" + dataset + ".root"
if box:
    out_path = "trees/box_selected/MaxDigis_tree_" + dataset + ".root"
else:
    out_path = "trees/MaxDigis_tree_" + dataset + ".root"

f = TFile( out_path, 'RECREATE' )
comp_tree = TTree( 'comparator', 'Max Comparator digis in a single chamber in each ring per event' )
wire_tree = TTree( 'wire', 'Max Comparator digis in a single chamber in each ring per event' )
# tree.Branch( 'digis', digis, 'Ev_max_nComp_ME11/I:Ev_max_nComp_ME12:Ev_max_nComp_ME13:Ev_max_nComp_ME21:Ev_max_nComp_ME22:Ev_max_nComp_ME31:Ev_max_nComp_ME32:Ev_max_nComp_ME41:Ev_max_nComp_ME42:Ev_max_nWire_ME11:Ev_max_nWire_ME12:Ev_max_nWire_ME13:Ev_max_nWire_ME21:Ev_max_nWire_ME22:Ev_max_nWire_ME31:Ev_max_nWire_ME32:Ev_max_nWire_ME41:Ev_max_nWire_ME42',0 )
comp_tree.Branch( 'Ev_max_nComp_ME11', comp_ME11, 'comp_ME11/F')
comp_tree.Branch( 'Ev_max_nComp_ME12', comp_ME12, 'comp_ME12/F')
comp_tree.Branch( 'Ev_max_nComp_ME13', comp_ME13, 'comp_ME13/F')
comp_tree.Branch( 'Ev_max_nComp_ME21', comp_ME21, 'comp_ME21/F')
comp_tree.Branch( 'Ev_max_nComp_ME22', comp_ME22, 'comp_ME22/F')
comp_tree.Branch( 'Ev_max_nComp_ME31', comp_ME31, 'comp_ME31/F')
comp_tree.Branch( 'Ev_max_nComp_ME32', comp_ME32, 'comp_ME32/F')
comp_tree.Branch( 'Ev_max_nComp_ME41', comp_ME41, 'comp_ME41/F')
comp_tree.Branch( 'Ev_max_nComp_ME42', comp_ME42, 'comp_ME42/F')

wire_tree.Branch( 'Ev_max_nWire_ME11', wire_ME11, 'wire_ME11/F')
wire_tree.Branch( 'Ev_max_nWire_ME12', wire_ME12, 'wire_ME12/F')
wire_tree.Branch( 'Ev_max_nWire_ME13', wire_ME13, 'wire_ME13/F')
wire_tree.Branch( 'Ev_max_nWire_ME21', wire_ME21, 'wire_ME21/F')
wire_tree.Branch( 'Ev_max_nWire_ME22', wire_ME22, 'wire_ME22/F')
wire_tree.Branch( 'Ev_max_nWire_ME31', wire_ME31, 'wire_ME31/F')
wire_tree.Branch( 'Ev_max_nWire_ME32', wire_ME32, 'wire_ME32/F')
wire_tree.Branch( 'Ev_max_nWire_ME41', wire_ME41, 'wire_ME41/F')
wire_tree.Branch( 'Ev_max_nWire_ME42', wire_ME42, 'wire_ME42/F')

if dataset=="Data_pre-selected":
    comp_pass_list = open("passlists/comp_pass_list.txt","w")
    wire_pass_list = open("passlists/wire_pass_list.txt","w")
    comp_last_text = ""
    wire_last_text = ""
    compXX = [98, 56, 30, 49, 42, 49, 35, 42, 31]
    wireXX = [104, 92, 32, 133, 83, 130, 74, 127, 88]
    comp_pass_count = 0
    wire_pass_count = 0

In [5]:
if dataset=="Data" or dataset=="Data_pre-selected":
  MC = False
  Neutrino = False
elif dataset=="Neutrino_gun" or dataset=="Nu_E10-pythia8-gun":
  MC = False
  Neutrino = True
else:
  MC = True
  Neutrino = False

if MC:
  File = TFile(in_path,"READ")
elif Neutrino:
  File = TFile("/uscms/home/menendez/nobackup/Trigger/CMSSW_10_6_4/src/Data/TPEHists_Neutrino_gun.root","READ")
else:
  #File = TFile("/uscms/home/menendez/nobackup/Trigger/CMSSW_10_6_4/src/Data/TPEHists_Data.root","READ")
  File = TFile(in_path,"READ")
clct = File.Get("lctreader/Ev_clcttree")
alct = File.Get("lctreader/Ev_alcttree")
llp = File.Get("lctreader/llp")

nEntries_clct = clct.GetEntries()
nEntries_alct = alct.GetEntries()
if MC:
  nEntries_llp = llp.GetEntries()

In [6]:
llp_accept = []
#max_nComp = np.array([0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0])
max_nComp = np.zeros((5,4))
max_nWire = np.zeros((5,4))
lastEventclct = 0
lastEventalct = 0

In [7]:
#Used for box selection
zMin = [4.5,5.7,5.8,7,7,8,8,9,9]
zMax = [5.5,6.7,6.8,8,8,9,9,10,10]
rMin = [1.0,2.5,4.7,1.4,3.5,1.6,3.5,1.8,3.5]
rMax = [2.8,4.5,6.8,3.5,7.0,3.5,7.0,3.5,7.0]

In [8]:
def get_R(x,y):
    R = np.sqrt(np.square(x) + np.square(y))
    return R

In [9]:
passed=0
if MC:
    print "Finding LLP Acceptance"
    for i in tqdm(range(0, nEntries_llp)):
        llp.GetEntry(i)

        if(llp.llp_in_acceptance[0]==1 or llp.llp_in_acceptance[1]==1):
            if box:
                for nlp in range(0,2):
                    R = get_R((llp.llp_decay_x[nlp]/100),(llp.llp_decay_y[nlp]/100))
                    for cut in range(0,len(zMin)):
                        if(abs(llp.llp_decay_z[nlp]/100) > zMin[cut] and abs(llp.llp_decay_z[nlp]/100) < zMax[cut] and R > rMin[cut] and R < rMax[cut]):
                            llp_accept.append(llp.event)
                            passed+=1
                            break
            else:
                llp_accept.append(llp.event)
                passed+=1
                        
print "# passed = " + str(passed)

# passed = 0


In [10]:
print "Starting CLCT Analysis"
for i in tqdm(range(0, nEntries_clct)):
  clct.GetEntry(i)

  if MC:
    if clct.t_Event not in llp_accept:
      continue
    
  if(clct.t_Event!=lastEventclct):
    lastEventclct = clct.t_Event
    
    comp_ME11[0] = max_nComp[1][1]
    comp_ME12[0] = max_nComp[1][2]
    comp_ME13[0] = max_nComp[1][3]
    comp_ME21[0] = max_nComp[2][1]
    comp_ME22[0] = max_nComp[2][2]
    comp_ME31[0] = max_nComp[3][1]
    comp_ME32[0] = max_nComp[3][2]
    comp_ME41[0] = max_nComp[4][1]
    comp_ME42[0] = max_nComp[4][2]
    max_nComp = np.zeros((5,4))

    comp_tree.Fill()
       
        
  if clct.t_nComp > max_nComp[clct.t_station][clct.t_ring]:
    max_nComp[clct.t_station][clct.t_ring] = clct.t_nComp
   
  if dataset=="Data_pre-selected":
    if (max_nComp[1][1]>compXX[0] or max_nComp[1][2]>compXX[1] or max_nComp[1][3]>compXX[2] or
        max_nComp[2][1]>compXX[3] or max_nComp[2][2]>compXX[4] or
        max_nComp[3][1]>compXX[5] or max_nComp[3][2]>compXX[6] or
        max_nComp[4][1]>compXX[7] or max_nComp[4][2]>compXX[8]):
      
        comp_pass_text = "%i:%i:%d\n" % (clct.t_Run,clct.t_Lumi,clct.t_Event)
        if comp_pass_text!=comp_last_text:
          comp_pass_list.write(comp_pass_text)
          comp_pass_count+=1
        comp_last_text = comp_pass_text

if dataset=="Data_pre-selected":
    print "%i events passed cut" %(comp_pass_count)

Starting CLCT Analysis


100%|██████████| 275018/275018 [00:12<00:00, 22443.67it/s]

2810 events passed cut


In [11]:
print "Starting ALCT Analysis"
for i in tqdm(range(0, nEntries_alct)):
  alct.GetEntry(i)

  if MC:
    if alct.t_Event not in llp_accept:
      continue
    
  if(alct.t_Event!=lastEventalct):
    lastEventalct = alct.t_Event
    
    wire_ME11[0] = max_nWire[1][1]
    wire_ME12[0] = max_nWire[1][2]
    wire_ME13[0] = max_nWire[1][3]
    wire_ME21[0] = max_nWire[2][1]
    wire_ME22[0] = max_nWire[2][2]
    wire_ME31[0] = max_nWire[3][1]
    wire_ME32[0] = max_nWire[3][2]
    wire_ME41[0] = max_nWire[4][1]
    wire_ME42[0] = max_nWire[4][2]
    max_nWire = np.zeros((5,4))

    wire_tree.Fill()
  

  if alct.t_nWire > max_nWire[alct.t_station][alct.t_ring]:
    max_nWire[alct.t_station][alct.t_ring] = alct.t_nWire
    
  if dataset=="Data_pre-selected":
    if (max_nWire[1][1]>wireXX[0] or max_nWire[1][2]>wireXX[1] or max_nWire[1][3]>wireXX[2] or
        max_nWire[2][1]>wireXX[3] or max_nWire[2][2]>wireXX[4] or
        max_nWire[3][1]>wireXX[5] or max_nWire[3][2]>wireXX[6] or
        max_nWire[4][1]>wireXX[7] or max_nWire[4][2]>wireXX[8]):
        
      wire_pass_text = "%i:%i:%d\n" % (alct.t_Run,alct.t_Lumi,alct.t_Event)
      if wire_pass_text!=wire_last_text:
        wire_pass_list.write(wire_pass_text)
        wire_pass_count+=1
      wire_last_text = wire_pass_text
    
if dataset=="Data_pre-selected":
    print "%i events passed cut" %(wire_pass_count)

  1%|          | 2141/326110 [00:00<00:15, 21407.62it/s]

Starting ALCT Analysis


100%|██████████| 326110/326110 [00:14<00:00, 23093.24it/s]

444 events passed cut


In [12]:
f.Write()
f.Close()
if dataset=="Data_pre-selected":
    comp_pass_list.close()
    wire_pass_list.close()